## XGBoost y predicción de compra
Hoy es sábado. Un bello día para introducir un nuevo modelo de _machine learning_. Hoy vamos a conocer _XGBoost_, uno de los algoritmos más efectivos y potentes en aprendizaje automático. A diferencia de los modelos con los que hemos trabajado antes, _XGBoost_ combina árboles y optimización para ser rápido y preciso. Vamos a aplicarlo para predecir si un cliente realizará una compra en función de sus datos personales y comportamiento histórico.

## Creación de los datos
Sí, _you know the drill_. Vamos a crear los datos. En otros proyectos usaremos más datos reales.

In [9]:
# Ya estamos bien familiarizados con la creación de datos para analizarlos.
import numpy as np
import pandas as pd

np.random.seed(52)
n = 1000
df = pd.DataFrame({
    'edad': np.random.randint(18, 70, n),
    'ingresos': np.random.normal(35000, 12000, n).astype(int),
    'antiguedad': np.random.randint(1, 12, n),
    'historial_compras': np.random.poisson(4, n)
})

# Variable objetivo binaria: compra (1) o no (0) con base en los datos, para que sea más realista.
df['compra'] = ((df['historial_compras'] > 6) | (df['ingresos'] > 45000)) & (df['edad'] < 50)
df['compra'] = df['compra'].astype(int)
df.head()

,edad,ingresos,antiguedad,historial_compras,compra
0,46,51506,10,4,1
1,29,54241,2,4,1
2,31,50773,11,1,1
3,41,20746,3,4,0
4,40,36328,5,6,0


## Preparamos los datos
Dividimos los datos en entrenamiento y prueba, y escalamos variables para igualar rangos. Ya sabes, nuestro `StandardScaler` de vuelta a la acción.

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop('compra', axis=1)
y = df['compra']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## ¿Qué es XGBoost?
Es un algoritmo de boosting basado en árboles (así como _Random Forest_), que funciona corrigiendo errores de modelos débiles previos para construir uno fuerte. Destaca por su precisión, velocidad y capacidad para manejar datos complejos y no lineales. Es robusto.

Su fortaleza está en cómo conjuga muchos árboles de decisión débiles y los entrena de forma secuencial: cada árbol nuevo se enfoca en los errores cometidos por los árboles previos, afinando así el resultado final.

Por si lo anterior no fuera suficiente, es también un modelo flexible para tareas de clasificación y regresión. Precisión y flexibilidad. ¿Qué más necesitas?

A diferencia de _Random Forest_, el modelo que utilizamos en los dos cuadernos anteriores, _XGBoost_ trabaja de manera secuencial. Cada árbol intenta corregir los errores de los anteriores. _Random Forest_ crea árboles independiente entre sí y los entrena en paralelo usando diferentes muestras de los datos (_bagging_). _XPGBoost_ es considerado un modelo más preciso que _Random Forest_ cuando se trata de datos muy complejos.

## Entrenamos el modelo y hacemos predicciones
Tenemos que instalar la librería `xgboost`, la cual no se puede importar directamente en _JupyterHub_.

In [7]:
import sys
!{sys.executable} -m pip install xgboost


import xgboost as xgb

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=0)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)


   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/72.0 MB 1.0 MB/s eta 0:01:09
   ---------------------------------------- 0.8/72.0 MB 1.1 MB/s eta 0:01:04
    --------------------------------------- 1.0/72.0 MB 1.1 MB/s eta 0:01:03
    --------------------------------------- 1.3/72.0 MB 1.2 MB/s eta 0:00:58
    --------------------------------------- 1.6/72.0 MB 1.3 MB/s eta 0:00:57
   - -------------------------------------- 2.1/72.0 MB 1.3 MB/s eta 0:00:52
   - -------------------------------------- 2.4/72.0 MB 1.4 MB/s eta 0:00:50
   - -------------------------------------- 2.9/72.0 MB 1.5 MB/s eta 0:00:47
   - -------------------------------------- 3.1/72.0 MB 1.5 MB/s eta 0:00:46
   -- ------------------------------------- 3.7/72.0 MB 1.6 MB/s eta 0:00:44
   -- --------------

C:\Users\rober\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [16:40:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


## Evaluación de su precisión
Veremos la precisión y la matriz de confusión. Exactamente las mismas métricas de desempeño que en el cuaderno anterior.

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Precisión:", accuracy_score(y_test, y_pred))
print()
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

Precisión: 1.0

Matriz de confusión:
[[165   0]
 [  0  35]]


## Conclusión
- Como puedes ver su precisión fue perfecta. Tenemos 1. No ha habido errores.
- La matriz de confusión nos muestra lo mismo: acertó en todos los valores negativos, que eran 165. Y acertó en todos los valores positivos, que eran 35. Precisión absoluta.

Así que recuerda, _XGBoost_ es un arma potente que combina árboles para superar las limitaciones de modelos individuales. Perfecto para problemas de clasificación con gran cantidad de datos.